In [1]:
import pandas as pd
import numpy as np



In [2]:
df = pd.read_csv('SPAM text message 20170820 - Data.csv')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
df.isnull().sum()

Category    0
Message     0
dtype: int64

In [7]:
df.shape

(5572, 2)

In [3]:
x = df.Message
y = df.Category

# split into train and test
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.20, random_state = 98, stratify = y)

print(f'xtrain: {xtrain.shape}')
print(f'xtest: {xtest.shape}')
print('ytrain: {}'.format(ytrain.shape))
print('ytest: {}'.format(ytest.shape))

xtrain: (4457,)
xtest: (1115,)
ytrain: (4457,)
ytest: (1115,)


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import ComplementNB, MultinomialNB
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

In [5]:
pipeCNB = Pipeline([("vect", TfidfVectorizer()),("clf", ComplementNB())])
pipelMNB = Pipeline([("vect", TfidfVectorizer()),("clf",MultinomialNB())])
pipelSVC = Pipeline([("vect", TfidfVectorizer()),("clf",SVC())])

In [6]:
pipeCNB.fit(xtrain, ytrain)
predictCNB = pipeCNB.predict(xtest)
print(f'CNB:{accuracy_score(ytest,predictCNB): .2f}')
pipelMNB.fit(xtrain, ytrain)
predictMNB = pipelMNB.predict(xtest)
print(f'NNB:{accuracy_score(ytest,predictMNB): .2f}')
pipelSVC.fit(xtrain, ytrain)
predictSVC = pipelSVC.predict(xtest)
print(f'SVC:{accuracy_score(ytest,predictSVC): .2f}')

CNB: 0.99
NNB: 0.97
SVC: 0.99


In [7]:
print(classification_report(ytest,predictSVC)) 

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99       966
        spam       1.00      0.92      0.96       149

    accuracy                           0.99      1115
   macro avg       0.99      0.96      0.98      1115
weighted avg       0.99      0.99      0.99      1115



In [8]:
print(classification_report(ytest,predictCNB)) 

              precision    recall  f1-score   support

         ham       0.99      0.99      0.99       966
        spam       0.96      0.93      0.95       149

    accuracy                           0.99      1115
   macro avg       0.97      0.96      0.97      1115
weighted avg       0.99      0.99      0.99      1115



In [9]:
pipelSVC.predict(['you have won a $1000 prize contact us for the reward','hello, hope you are good?'])

array(['spam', 'ham'], dtype=object)

In [10]:
pipelSVC.classes_

array(['ham', 'spam'], dtype=object)

In [11]:
import pickle
pickle.dump(pipelSVC, open('model.pkl', 'wb'))

In [12]:
model = pickle.load(open('model.pkl', 'rb'))

In [13]:
model.predict(['you have won a $1000 prize contact us for the reward','hello, hope you are good?'])

array(['spam', 'ham'], dtype=object)

In [14]:
newcategory = df['Message'].sample(5)
newcategory

803     FREE for 1st week! No1 Nokia tone 4 ur mobile ...
2362    Hi. I'm sorry i missed your call. Can you pls ...
2271                           U know we watchin at lido?
782     Hmmm ... I thought we said 2 hours slave, not ...
3560    URGENT! We are trying to contact U Todays draw...
Name: Message, dtype: object

In [15]:
model.predict(newcategory)

array(['spam', 'ham', 'ham', 'ham', 'spam'], dtype=object)

In [16]:
# batch predict for the company, and save the predicted customer into a new column called predicted customer
newcategory['predcat'] = model.predict(newcategory)
newcategory

803        FREE for 1st week! No1 Nokia tone 4 ur mobile ...
2362       Hi. I'm sorry i missed your call. Can you pls ...
2271                              U know we watchin at lido?
782        Hmmm ... I thought we said 2 hours slave, not ...
3560       URGENT! We are trying to contact U Todays draw...
predcat                          [spam, ham, ham, ham, spam]
Name: Message, dtype: object